<a href="https://colab.research.google.com/github/sricinu/upgrad_rag/blob/main/Semantic_Search_Practice_Exercise_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import sentence_transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_path = '/content/drive/My Drive/HelpMate AI Codes/HelpMate Practice Question/'. # Update as per your path

In [ ]:
job_postings = pd.read_csv(input_path + 'job_postings.csv')

In [ ]:
job_postings

,title,description,location,work_type
0,Licensed Insurance Agent,While many industries were hurt by the last fe...,"Chico, CA",FULL_TIME
1,Sales Manager,Are you a dynamic and creative marketing profe...,"Santa Clarita, CA",FULL_TIME
2,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,"New York, NY",CONTRACT
3,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,"Forney, TX",FULL_TIME
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,"New York, NY",FULL_TIME
...,...,...,...,...
495,Electrical Engineer - P&C Design,Job Description:Role: Electrical Engineer - P&...,United States,FULL_TIME
496,Editor,Role : Editor Remote Description: The Spotify ...,"New York, NY",CONTRACT
497,Spiritual Care Coordinator,"Coordinates spiritual, emotional and psycholog...","Fremont, OH",FULL_TIME
498,Supply Chain Intern,Learn and assist with all activities associate...,"Fort Wayne, IN",INTERNSHIP


In [ ]:
# Load the embedding model

MiniLM_model_name = "all-MiniLM-L6-v2"
MiniLM_model = SentenceTransformer(MiniLM_model_name)

In [ ]:
# Load the embedding model

QA_model_name = "multi-qa-MiniLM-L6-cos-v1"
QA_model = SentenceTransformer(QA_model_name)

In [ ]:
# Function to generate embeddings for text.

def generate_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings

In [ ]:
# Call the function to generate embeddings. It will take upto 5 minutes to embed all 500 entries in the dataset.

job_postings['Embeddings_MiniLM'] = job_postings['description'].apply(lambda x: generate_embeddings(MiniLM_model, [x])[0])
job_postings['Embeddings_QA'] = job_postings['description'].apply(lambda x: generate_embeddings(QA_model, [x])[0])

In [ ]:
job_postings

,title,description,location,work_type,Embeddings_MiniLM,Embeddings_QA
0,Licensed Insurance Agent,While many industries were hurt by the last fe...,"Chico, CA",FULL_TIME,"[tensor(-0.0357), tensor(0.0185), tensor(-0.08...","[tensor(0.0241), tensor(0.0494), tensor(-0.033..."
1,Sales Manager,Are you a dynamic and creative marketing profe...,"Santa Clarita, CA",FULL_TIME,"[tensor(0.0146), tensor(-0.0772), tensor(0.003...","[tensor(0.0172), tensor(-0.0164), tensor(0.008..."
2,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,"New York, NY",CONTRACT,"[tensor(0.0594), tensor(-0.0626), tensor(-0.05...","[tensor(0.0255), tensor(-0.0302), tensor(-0.08..."
3,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,"Forney, TX",FULL_TIME,"[tensor(0.0161), tensor(0.0097), tensor(0.0055...","[tensor(0.0107), tensor(0.0243), tensor(-0.029..."
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,"New York, NY",FULL_TIME,"[tensor(0.0196), tensor(-0.1300), tensor(0.058...","[tensor(0.0156), tensor(-0.0645), tensor(0.006..."
...,...,...,...,...,...,...
495,Electrical Engineer - P&C Design,Job Description:Role: Electrical Engineer - P&...,United States,FULL_TIME,"[tensor(0.0284), tensor(0.0051), tensor(0.0042...","[tensor(-0.0445), tensor(0.0474), tensor(0.048..."
496,Editor,Role : Editor Remote Description: The Spotify ...,"New York, NY",CONTRACT,"[tensor(0.0469), tensor(-0.1052), tensor(-0.02...","[tensor(0.0431), tensor(-0.0604), tensor(0.051..."
497,Spiritual Care Coordinator,"Coordinates spiritual, emotional and psycholog...","Fremont, OH",FULL_TIME,"[tensor(0.0375), tensor(-0.0017), tensor(-0.03...","[tensor(0.0759), tensor(0.0399), tensor(-0.082..."
498,Supply Chain Intern,Learn and assist with all activities associate...,"Fort Wayne, IN",INTERNSHIP,"[tensor(-0.0180), tensor(0.0153), tensor(0.005...","[tensor(0.0025), tensor(0.0193), tensor(-0.033..."


In [ ]:
# query_1 = I am looking for a data engineer job that offers competitive pay
# query_2 = I want to work in UI design in the gaming industry

In [ ]:
query1 = input()

I am looking for a data engineer job that offers competitive pay


In [ ]:
query2 = input()

I want to work in UI design in the gaming industry


In [ ]:
# Define the function for calculating cosine similarity

def calculate_similarity(embedding1, embedding2):
    cosine_score = util.pytorch_cos_sim(embedding1, embedding2)
    # Convert the result to a Python float
    similarity = cosine_score.item()

    return similarity

In [ ]:
# Calculate the query embedding for query1
query_embedding_miniLM_1 = MiniLM_model.encode(query1, convert_to_tensor=True)
query_embedding_QA_1 = QA_model.encode(query1, convert_to_tensor=True)


# Calculate the query embedding for query2
query_embedding_miniLM_2 = MiniLM_model.encode(query2, convert_to_tensor=True)
query_embedding_QA_2 = QA_model.encode(query2, convert_to_tensor=True)

In [ ]:
# Calculate similarity scores between the query embedding and all chunk embeddings for query1
job_postings['Similarity_1_miniLM'] = job_postings['Embeddings_MiniLM'].apply(lambda x: calculate_similarity(query_embedding_miniLM_1, x))
job_postings['Similarity_1_QA'] = job_postings['Embeddings_QA'].apply(lambda x: calculate_similarity(query_embedding_QA_1, x))


# Calculate similarity scores between the query embedding and all chunk embeddings for query2
job_postings['Similarity_2_miniLM'] = job_postings['Embeddings_MiniLM'].apply(lambda x: calculate_similarity(query_embedding_miniLM_2, x))
job_postings['Similarity_2_QA'] = job_postings['Embeddings_QA'].apply(lambda x: calculate_similarity(query_embedding_QA_2, x))

In [ ]:
# Extract top 3 results for query1 for both models

top_3_MiniLM_query1 = job_postings.sort_values(by='Similarity_1_miniLM', ascending=False)[:3]
top_3_QA_query1 = job_postings.sort_values(by='Similarity_1_QA', ascending=False)[:3]

In [ ]:
# Extract top 3 results for query2 for both models

top_3_MiniLM_query2 = job_postings.sort_values(by='Similarity_2_miniLM', ascending=False)[:3]
top_3_QA_query2 = job_postings.sort_values(by='Similarity_2_QA', ascending=False)[:3]

In [ ]:
top_3_MiniLM_query1

,title,description,location,work_type,Embeddings_MiniLM,Embeddings_QA,Similarity_1_miniLM,Similarity_1_QA,Similarity_2_miniLM,Similarity_2_QA
266,Data Engineer,"Job Description:\n• Design, develop, and launc...",United States,CONTRACT,"[tensor(-0.0304), tensor(0.0321), tensor(0.022...","[tensor(-0.0350), tensor(0.0405), tensor(0.041...",0.529741,0.543093,0.175004,0.205815
396,Azure Data Engineer,Need Genuine candidates who has solid experien...,United States,CONTRACT,"[tensor(-0.0148), tensor(-0.0782), tensor(0.02...","[tensor(0.0092), tensor(-0.0536), tensor(-0.00...",0.478422,0.387438,0.299140,0.203843
440,Senior Data Engineer,Combine your technical expertise and problem-s...,"Virginia, United States",FULL_TIME,"[tensor(-0.0201), tensor(0.0069), tensor(-0.01...","[tensor(-0.0178), tensor(0.0046), tensor(0.000...",0.449332,0.342863,0.210983,0.186412


In [ ]:
top_3_QA_query1

,title,description,location,work_type,Embeddings_MiniLM,Embeddings_QA,Similarity_1_miniLM,Similarity_1_QA,Similarity_2_miniLM,Similarity_2_QA
266,Data Engineer,"Job Description:\n• Design, develop, and launc...",United States,CONTRACT,"[tensor(-0.0304), tensor(0.0321), tensor(0.022...","[tensor(-0.0350), tensor(0.0405), tensor(0.041...",0.529741,0.543093,0.175004,0.205815
292,Data Scientist/ Product Analyst,Looking for candidates with 4+ years’ experien...,"San Francisco, CA",CONTRACT,"[tensor(0.0009), tensor(-0.0532), tensor(0.044...","[tensor(0.0243), tensor(0.0120), tensor(0.0096...",0.309458,0.462681,0.116504,0.295949
287,Assistant to Sales Director,Please send resume with salary requirements an...,"Chicago, IL",FULL_TIME,"[tensor(-0.1007), tensor(-0.0512), tensor(-0.0...","[tensor(-0.0475), tensor(-0.0001), tensor(0.00...",0.215967,0.416805,0.243621,0.256875


In [ ]:
top_3_MiniLM_query2

,title,description,location,work_type,Embeddings_MiniLM,Embeddings_QA,Similarity_1_miniLM,Similarity_1_QA,Similarity_2_miniLM,Similarity_2_QA
126,UX Designer/Architect-Plano TX and Columbus OH...,Our client is looking for UX Designer.\nLocati...,"Plano, TX",CONTRACT,"[tensor(0.0440), tensor(0.0145), tensor(0.0494...","[tensor(0.0026), tensor(0.0201), tensor(0.0548...",0.317580,0.327876,0.472885,0.422205
307,Software Engineer- Graphics Designer,Job Description\nExperience with C/C++ program...,United States,FULL_TIME,"[tensor(-0.1053), tensor(0.0162), tensor(0.020...","[tensor(-0.0668), tensor(-0.0321), tensor(-0.0...",0.373739,0.296570,0.412720,0.389942
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,"New York, NY",FULL_TIME,"[tensor(0.0196), tensor(-0.1300), tensor(0.058...","[tensor(0.0156), tensor(-0.0645), tensor(0.006...",0.253242,0.296694,0.398065,0.369122


In [ ]:
top_3_QA_query2

,title,description,location,work_type,Embeddings_MiniLM,Embeddings_QA,Similarity_1_miniLM,Similarity_1_QA,Similarity_2_miniLM,Similarity_2_QA
438,Lead Game Designer,Position: Lead Game DesignerLocation: Remote W...,United States,FULL_TIME,"[tensor(0.0055), tensor(-0.0065), tensor(-0.02...","[tensor(-0.0034), tensor(0.0232), tensor(-0.03...",0.134383,0.265356,0.395483,0.532436
327,Head of Visual Design Operating Systems,STEALTH GAMING OPERATING SYSTEM. \nWe are a ve...,United States,FULL_TIME,"[tensor(-0.0058), tensor(-0.0010), tensor(-0.0...","[tensor(-0.0337), tensor(-0.0295), tensor(0.03...",0.198510,0.224083,0.366534,0.519794
267,Sr. AV Designer,"ABOUT THE POSITION\nAs a Senior AV Designer, y...","West Valley City, UT",FULL_TIME,"[tensor(-0.0371), tensor(0.0614), tensor(-0.02...","[tensor(0.0251), tensor(0.0615), tensor(-0.028...",0.243018,0.256798,0.396645,0.432892
